In [1]:
import numpy as np
import time
import pickle
## I import sys to kill the program if an option is not correct.
import sys
import os
import csv

from RhoAndBeta import CalcRhoAndBetaVectors
from UtilitiesOptimization import SubgrAlgSavPrimDualObjInd, \
    SubgrAlgSavPrimDualObjFn_L2Ind
from SimulationCode import ExpPareto
from Utilities import CreateTableParetoInd, CreateTableParetoL2_L2Ind_Gr

In [2]:
## Read the data created in CreateDataJupNot
suffix='InstanceInfo/Ins1LongRun/'
num_impressions=pickle.load(open(suffix+'num_impressions'+'.p',"rb"))
numCampaigns=pickle.load(open(suffix+'numCampaigns'+'.p',"rb"))
num_edges=pickle.load(open(suffix+'num_edges'+'.p',"rb"))
index_Imps=pickle.load(open(suffix+'index_Imps'+'.p',"rb"))
index_sizeCamps=pickle.load(open(suffix+'index_sizeCamps'+'.p',"rb"))
index_startCamp=pickle.load(open(suffix+'index_startCamp'+'.p',"rb"))
vector_maxbid=pickle.load(open(suffix+'vector_maxbid'+'.p',"rb"))
vector_r=pickle.load(open(suffix+'vector_r'+'.p',"rb"))
vector_s=(pickle.load(open(suffix+'vector_s'+'.p',"rb"))).astype(int)
ext_s=pickle.load(open(suffix+'ext_s'+'.p',"rb"))
adverPerImp=(pickle.load(open(suffix+'adverPerImp'+'.p',"rb"))).astype(int)
UB_bidsPerImp = pickle.load(open(suffix+'UB_bids'+'.p',"rb"))
vector_ctr=pickle.load(open(suffix+'vector_ctr'+'.p',"rb"))
vector_rctr=pickle.load(open(suffix+'vector_rctr'+'.p',"rb"))
probImp=pickle.load(open(suffix+'probImp'+'.p',"rb"))

In [3]:
## In this Experiment we fix a budget to use. In this case we choose 100
vector_m = np.ones(numCampaigns)*100

In [4]:
## If this parameter is true then first price auctions (\beta_i(b) =b) are used, otherwise second price.
firstPrice = True
## If this parameter is true a parameterSearch will be performd and .csv will be made,
parameterSearch = False
## We use $\tau_k = 1/m_k$
expForTau=-1.0
tau = np.power(vector_m,expForTau)
UB_bids = UB_bidsPerImp[index_Imps]

suffix='ResultsPareto/'
current_directory = os.getcwd()
results_directory = os.path.join(current_directory, suffix)
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Parameter Search

This parameter search looks for a constant $C$, such that the dual method shows 
empirical convergence. Given that here we only look for training convergence, we don't 
need to cross-validate or similar.

In [5]:
if parameterSearch:
    input_var = input("This will execute a simple parameter search.\
    If you just wanted to run just a full long run do parameterSearch= False  \
    and kill this execution. To continue press enter.")
    c=np.array([1.0,0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001])
    p_grad_Type = 0
    num_it = 1000
    it_per_cal = 250
    init_lam=np.zeros((numCampaigns))
    alphas_pure=np.fromfunction(lambda i, j: (1/(np.sqrt(i + 1))), \
        (num_it, 1), dtype=int)[:,0] 
    
#     dualObjFn, primalObjFn, dualObjFnAvg, primalObjFnAvg, budget_used, \
#     budget_LamAvgUse, dual_vars, dual_varsAvg = [], [], [], [], [], [], [], []

    nameResults='ParameterSearch'+'It_'+str(num_it)
    f = open(suffix+nameResults+'.csv', 'wt')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow( ('Auction Type', 'Problem Type', 'Ite', 'Cte', 'DualFnValue',\
        'PrimalValue','DualFnValueAvg','PrimalFnValueAvg') )
    
    print('Using First Price Auctions')
    firstPrice = True
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))

            
    print('Using Second Price Auctions')
    firstPrice = False
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
    f.close()

# The following value are obtained just by Looking at The Parameter Search .csv

In [6]:
## Best constants
consBestFP = [0.0001, 0.5, 0.5]
consBestSP = [0.0001, 0.5, 0.5]

### Run Pareto

In [7]:
num_itInd, num_itL2, num_itL2Ind = 10000, 10000, 10000
init_lam = np.zeros(numCampaigns)

shuffle = False
sim = 100

np.random.seed(12345)
vecOfSeeds = np.random.randint(100000, size=sim)

In [8]:
multTP = [0.018, 0.020, 0.022, 0.025, 0.027, 0.030, 0.033, 0.037, 0.041, \
    0.045, 0.050, 0.055, 0.061, 0.067, 0.074, 0.082, 0.091, 0.100, 0.111, \
    0.122, 0.135, 0.150, 0.165, 0.183, 0.202, 0.223, 0.247, 0.273, 0.301, \
    0.333, 0.368, 0.407, 0.449, 0.497, 0.549, 0.607, 0.670, 0.741, 0.819, \
    0.905, 1.000, 1.105, 1.221, 1.350, 1.492, 1.649, 1.822, 2.014, 2.226, \
    2.460]

multGr = [np.round(0.4+0.02*y, decimals = 2) for y in range(50)]

p_grad_TypeInd, p_grad_TypeL2, p_grad_TypeL2Ind = 0, 1, 2

alphasInd, alphasL2, alphasL2Ind = 0, 0, 0

if firstPrice:
    alphasInd = consBestFP[0] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itInd)])
    alphasL2 = consBestFP[1] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itL2)])
    alphasL2Ind = consBestFP[2] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itL2Ind)])
else:
    alphasInd = consBestSP[0] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itInd)])
    alphasL2 = consBestSP[1] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itL2)])
    alphasL2Ind = consBestSP[2] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itL2Ind)])


In [9]:
listToRetInd, dictToRetL2, dictToRetL2Ind, dictToRetGr = ExpPareto(numCampaigns, num_impressions,\
    num_edges, index_Imps, index_sizeCamps, vector_s, vector_r, vector_m, vector_ctr, vector_rctr, \
    UB_bidsPerImp, adverPerImp, alphasInd, num_itInd, alphasL2, num_itL2, alphasL2Ind, num_itL2Ind, \
    p_grad_TypeInd, p_grad_TypeL2, p_grad_TypeL2Ind, multGr, multTP, init_lam, sim, firstPrice, \
    vecOfSeeds = vecOfSeeds, shuffle = shuffle)

We will first run the indicator case. 
Finishing running the Indicator part. It took: 393.68615889549255 secs.
Greedy multiplier: 0.4, L2/L2Ind multiplier: 0.018
Finished running the Primal-Dual Algorithms, it took: 148.2300980091095 secs.
Running the simulations took 711.5500078201294 secs.
Greedy multiplier: 0.42, L2/L2Ind multiplier: 0.02
Finished running the Primal-Dual Algorithms, it took: 148.8249990940094 secs.
Running the simulations took 703.9536650180817 secs.
Greedy multiplier: 0.44, L2/L2Ind multiplier: 0.022
Finished running the Primal-Dual Algorithms, it took: 148.71444821357727 secs.
Running the simulations took 703.3459310531616 secs.
Greedy multiplier: 0.46, L2/L2Ind multiplier: 0.025
Finished running the Primal-Dual Algorithms, it took: 149.0423300266266 secs.
Running the simulations took 704.2968001365662 secs.
Greedy multiplier: 0.48, L2/L2Ind multiplier: 0.027
Finished running the Primal-Dual Algorithms, it took: 147.93511605262756 secs.
Running the simulations too

Finished running the Primal-Dual Algorithms, it took: 147.01215505599976 secs.
Running the simulations took 758.8736913204193 secs.
Greedy multiplier: 1.3, L2/L2Ind multiplier: 1.649
Finished running the Primal-Dual Algorithms, it took: 148.3853030204773 secs.
Running the simulations took 759.7908780574799 secs.
Greedy multiplier: 1.32, L2/L2Ind multiplier: 1.822
Finished running the Primal-Dual Algorithms, it took: 147.90110397338867 secs.
Running the simulations took 760.3537368774414 secs.
Greedy multiplier: 1.34, L2/L2Ind multiplier: 2.014
Finished running the Primal-Dual Algorithms, it took: 147.5442328453064 secs.
Running the simulations took 761.9487202167511 secs.
Greedy multiplier: 1.36, L2/L2Ind multiplier: 2.226
Finished running the Primal-Dual Algorithms, it took: 147.02785468101501 secs.
Running the simulations took 763.2927179336548 secs.
Greedy multiplier: 1.38, L2/L2Ind multiplier: 2.46
Finished running the Primal-Dual Algorithms, it took: 147.91480231285095 secs.
Runni

In [10]:
from Utilities import CreateTableSensitivity

nameToSaveFP = 'TableParetoLogFP.csv'
nameToSaveIndFP = 'TableParetoIndLogFP.csv'

TableParetoFP = CreateTableParetoL2_L2Ind_Gr(dictToRetL2, dictToRetL2Ind, dictToRetGr,\
                                             vector_m, multTP, multGr, sim)

TableJustIndFP = CreateTableParetoInd(listToRetInd, vector_m, sim, name = "FP")


with open(suffix+nameToSaveFP, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['multL2', 'multGr', 'sim',\
        'L2-Profit', 'L2-Revenue', 'L2-Cost', 'L2-BidsMade',\
        'L2-BidsWon', 'L2-ClicksWon', 'L2-%BudgetUsed',\
        'L2Ind-Profit', 'L2Ind-Revenue', 'L2Ind-Cost', 'L2Ind-BidsMade',\
        'L2Ind-BidsWon', 'L2Ind-ClicksWon', 'L2Ind-%BudgetUsed',\
        'Gr-Profit', 'Gr-Revenue', 'Gr-Cost', 'Gr-BidsMade',\
        'Gr-BidsWon', 'Gr-ClicksWon', 'Gr-%BudgetUsed'])
    [writer.writerow(r) for r in TableParetoFP]

with open(suffix+nameToSaveIndFP, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['sim', 'Ind-Profit', 'Ind-Revenue', 'Ind-Cost',\
        'Ind-BidsMade', 'Ind-BidsWon', 'Ind-ClicksWon', 'Ind-%BudgetUsed'])
    [writer.writerow(r) for r in TableJustIndFP]
    

    